In [15]:
import pandas as pd

# Define stock dataset info
stock_dataset_info = [
    ("AAPL", "../src/AAPL_historical_data.csv"),
    ("AMZN", "../src/AMZN_historical_data.csv"),
    ("GOOG", "../src/GOOG_historical_data.csv"),
    ("META", "../src/META_historical_data.csv"),
    ("MSFT", "../src/MSFT_historical_data.csv"),
    ("NVDA", "../src/NVDA_historical_data.csv"),
    ("TSLA", "../src/TSLA_historical_data.csv")
]

# Load stock datasets into a dictionary
stock_dataframes = {}
for stock_symbol, file_path in stock_dataset_info:
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])  # Convert date column
    df.set_index('Date', inplace=True)  # Set date as index
    stock_dataframes[stock_symbol] = df

# Load news headlines dataset
headlines_df = pd.read_csv('../src/raw_analyst_ratings.csv')  # Adjust the path as necessary

# Convert 'date' to datetime, coercing errors to NaT
headlines_df['date'] = pd.to_datetime(headlines_df['date'], errors='coerce')

# Drop rows where 'date' is NaT
headlines_df.dropna(subset=['date'], inplace=True)

# Set 'date' as the index
headlines_df.set_index('date', inplace=True)

headlines_df.head()

,Unnamed: 0,headline,url,publisher,stock
date,,,,,
2020-06-05 10:30:54-04:00,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,A
2020-06-03 10:45:20-04:00,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,A
2020-05-26 04:30:07-04:00,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,A
2020-05-22 12:45:06-04:00,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,A
2020-05-22 11:38:59-04:00,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,A


Data Alignment

In [22]:
# Assuming you have already loaded headlines_df and stock_dataframes

# Ensure stock_df is timezone-naive
    # if stock_df.index.tz is not None:
    #     stock_df.index = stock_df.index.tz_localize(None)

    # Filter headlines for the current stock ticker
    ticker_headlines = headlines_df[headlines_df['headline'].str.contains(stock_symbol, na=False, case=False)]

    # Resample the filtered headlines to daily frequency, aggregating
    news_daily = ticker_headlines.resample('D').agg({'headline': ' '.join})

    # Drop rows where there are no headlines
    news_daily.dropna(subset=['headline'], inplace=True)

    # Ensure news_daily is timezone-naive
    if news_daily.index.tz is not None:
        news_daily.index = news_daily.index.tz_localize(None)

    # Merge stock DataFrame with news DataFrame
    merged_df = pd.merge(stock_df, news_daily, left_index=True, right_index=True, how='inner')
    aligned_data[stock_symbol] = merged_df

    # Display the headlines for the current stock symbol
    print("")
    print(f"Aligned data for {stock_symbol}:")
    print("")
    print(news_daily.head(100))  # Display the first 100 rows of the merged DataFrame
    print("")


AttributeError: 'NoneType' object has no attribute 'index'